In [9]:
import requests

def city_name_to_coordinates(city_name):
    # send to GET https://nominatim.openstreetmap.org/search?q=Beijing&format=json
    user_agent = "omni weather app"
    headers = {"User-Agent": user_agent}
    search_url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json"
    resp = requests.get(search_url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Failed to get coordinates: {resp.status_code}")
    data = resp.json()
    print(f"Coordinates for {city_name}: {data[0]['lat']}, {data[0]['lon']}")
    return data[0]['lon'], data[0]['lat']

lon, lat = city_name_to_coordinates("Beijing, CN")

Coordinates for Beijing, CN: 40.1906320, 116.4121440


In [11]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def get_weather_forecast(latitude, longitude):
    """
    Get daily weather forecast from the Open-Meteo API.

    Args:
        latitude (float): Latitude for the location.
        longitude (float): Longitude for the location.

    Returns:
        pandas.DataFrame: A DataFrame containing the daily weather forecast.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": [
            "weather_code",
            "temperature_2m_max",
            "temperature_2m_min",
            "apparent_temperature_max",
            "apparent_temperature_min",
            "sunrise",
            "sunset",
            "daylight_duration",
            "sunshine_duration",
            "uv_index_max",
            "uv_index_clear_sky_max",
            "precipitation_sum",
            "rain_sum",
            "showers_sum",
            "snowfall_sum",
            "precipitation_hours",
            "precipitation_probability_max",
        ],
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left",
        )
    }

    for i, column in enumerate(params["daily"]):
        daily_data[column] = daily.Variables(i).ValuesAsNumpy()

    return pd.DataFrame(data=daily_data)


# Use the coordinates from the previous cell, converting them to float
weather_df = get_weather_forecast(float(lat), float(lon))
print(weather_df)


                       date  weather_code  temperature_2m_max  \
0 2025-06-20 00:00:00+00:00           3.0           34.014999   
1 2025-06-21 00:00:00+00:00           1.0           35.114998   
2 2025-06-22 00:00:00+00:00           3.0           37.314999   
3 2025-06-23 00:00:00+00:00           0.0           38.814999   
4 2025-06-24 00:00:00+00:00           3.0           40.514999   
5 2025-06-25 00:00:00+00:00           3.0           40.664997   
6 2025-06-26 00:00:00+00:00          80.0           29.315001   

   temperature_2m_min  apparent_temperature_max  apparent_temperature_min  \
0           21.515001                 34.388252                 20.077639   
1           22.265001                 35.287754                 21.876543   
2           21.515001                 38.125809                 22.453049   
3           23.315001                 39.416679                 23.837694   
4           23.465000                 39.938587                 22.959349   
5           24.01

In [ ]:
# use nws api

nws_url = "https://api.weather.gov/points/{lat},{lon}"
nws_url = nws_url.format(lat=lat, lon=lon)
nws_response = requests.get(nws_url)
if nws_response.status_code != 200:
    raise Exception(f"Failed to get NWS data: {nws_response.status_code}")
nws_data = nws_response.json()

print

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
import pprint

search = GoogleSerperAPIWrapper(k=3)
results = search.results("Apple Inc.")
# load json
pprint.pprint(results['organic'])
# 




[{'link': 'https://www.apple.com/',
  'position': 1,
  'snippet': 'Discover the innovative world of Apple and shop everything '
             'iPhone, iPad, Apple Watch, Mac, and Apple TV, plus explore '
             'accessories, entertainment, ...',
  'title': 'Apple'},
 {'link': 'https://en.wikipedia.org/wiki/Apple_Inc.',
  'position': 2,
  'snippet': 'Apple Inc. is an American multinational corporation and '
             'technology company headquartered in Cupertino, California, in '
             'Silicon Valley.',
  'title': 'Apple Inc. - Wikipedia'},
 {'date': '4 days ago',
  'link': 'https://www.britannica.com/money/Apple-Inc',
  'position': 3,
  'snippet': 'American manufacturer of personal computers, smartphones, and '
             'tablet computers. Apple was the first successful personal '
             'computer company and ...',
  'title': 'Apple Inc. | History, Products, Headquarters, & Facts - '
           'Britannica'}]


In [20]:
from langchain_community.document_loaders import WebBaseLoader
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()
# load all the links in the results
links = [result['link'] for result in results['organic']]
loader = WebBaseLoader(links)
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.apple.com/', 'title': 'Apple', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\nApple\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nApple\n\nAppleStoreMaciPadiPhoneWatch\nVisionAirPodsTV & HomeEntertainmentAccessoriesSupport\n\n\n0+\n\n\n\n\n\n\n\n\xa0\n\n\xa0\n\n\n\n\n\nBuy Mac or\xa0iPad for\xa0college\n\n\n\n\n\n\n\n\n\nwith education savings\n\n\n\n\n\n\n\nChoose AirPods or an eligible accessory1\n\n\n\n\nShop\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\nCollege Students\nMac and iPad. Major. In any field.\n\nLearn more\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\nMacBook Air\nSky blue color.Sky high performance with M4.\n\nLearn more\nBuy\n\n\n\nBuilt for Apple Intelligence.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\niPad Air\nNow supercharged by the M3 chip.\n\nLearn more\nBuy\n\n\n\nBuilt for Apple Intelligence.\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\nMacBook Pro\nA work of smart.\n\nLearn more\nBuy\n\n\n\nBuilt for Apple\xa0Intelligence.\n\n\n\n\n\n\n\n\n\n\n\